In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px

<a id="model-def"></a>
# <a id='toc10_'></a>[<b><span style='color:darkorange'>Step 1.1 |</span><span style='color:#1E3A8A'> Preparing Data</span></b>](#toc0_)

In [3]:
# Wczytanie i przygotowanie danych (Boston Housing Dataset)
housing = fetch_california_housing()
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = pd.Series(housing.target, name='PRICE')

# Skalowanie danych
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Podział na zbiór treningowy i testowy
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Konwersja do tensorów
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# view(-1,1) zamienia tensor [N] mna [N,1] bo mse_loss potrzebuje 2D
# -1 mówi - nie martw się ile jest wiersz, zrób z tego kolumnę
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

train_ds = TensorDataset(X_train_tensor, y_train_tensor)
val_ds = TensorDataset(X_val_tensor,y_val_tensor)

batch_size = 64
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)


In [ ]:
train_ds[3] # Zwraca X i y dla 3 próbki train_ds[3] -> (X[3], y[3])

(tensor([-1.0149,  0.5849, -0.5764, -0.1327, -0.0066,  0.0889, -1.3773,  1.2277]),
 tensor([0.9340]))



## 🧠 Co to jest **batch**?

### 👉 **Batch** = porcja danych przetwarzana naraz przez model

Gdy uczysz sieć neuronową, możesz:
- trenować na **całym zbiorze danych na raz** (→ _batch gradient descent_) ❌ NIEOPTYMALNE
- trenować na **pojedynczych przykładach** (→ _stochastic gradient descent_) ❌ niestabilne
- trenować na **małych grupach (batchach)** danych (→ _mini-batch gradient descent_) ✅ NAJCZĘSTSZE


## 🔢 Przykład

Masz 1000 próbek (np. mieszkań):  
Jeśli `batch_size = 64`, to dane będą dzielone tak:

| Nr batcha | Indeksy próbek       |
|-----------|-----------------------|
| 1         | 0–63                  |
| 2         | 64–127                |
| ...       | ...                   |
| 16        | 960–999 (ostatni)     |

➡️ **Model widzi po 64 próbki na raz, a potem aktualizuje wagi.**


## 📦 Co robi ten kod?

```python
train_ds = TensorDataset(X_train_tensor, y_train_tensor)
val_ds = TensorDataset(X_val_tensor, y_val_tensor)
```

### ✅ `TensorDataset(...)`

To prosty kontener, który przechowuje:
- dane wejściowe (X)
- etykiety wyjściowe (y)

Każdy element `train_ds[i]` to tuple `(X[i], y[i])`.


```python
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64)
```

### ✅ `DataLoader(...)`

To **automat do dzielenia danych na batch'e**.   
- dzieli dane na batch'e

- może je przemieszać (shuffle=True)

- daje kolejne porcje (mini-batche) do treningu

| Parametr | Znaczenie |
|----------|-----------|
| `train_ds` | Z jakiego zbioru ma brać dane |
| `batch_size=64` | Ile próbek w jednym batchu |
| `shuffle=True` | Czy losować kolejność przed każdą epoką (ważne dla treningu) |


## 🔄 Co się dzieje w `train_loop(...)`

```python
for batch in train_loader:
    X, y = batch
    pred = model(X)
    ...
```

Tutaj **każdy `batch` to tuple (X, y)** z rozmiarem:

```python
X.shape = [64, liczba_cech]
y.shape = [64, 1]
```

Dlatego model trenuje się szybciej, stabilniej i wykorzystuje lepiej GPU/CPU.


## 🔍 Porównanie różnych `batch_size`:

| `batch_size` | Czas uczenia | Stabilność | Wymagania RAM/GPU |
|--------------|--------------|------------|--------------------|
| 1 (SGD)      | wolny        | chaotyczny | niski              |
| 16–64        | szybki       | stabilny   | średni             |
| 512–1024     | szybki       | mniej stabilny | wysoki         |

🧠 **64** to typowa wartość startowa.







### ✅ batch = tuple dwóch tensorów:

| Nazwa | Typ | Rozmiar |
|-------|-----|---------|
| `X`   | `torch.Tensor` | `[batch_size, liczba_cech]` |
| `y`   | `torch.Tensor` | `[batch_size, 1]` lub `[batch_size]` |


### 🧠 Analogicznie:

| Kod | Co zwraca |
|-----|-----------|
| `train_ds[0]` | `(X[0], y[0])` — pojedyncza próbka |
| `next(iter(train_loader))` | `(X_batch, y_batch)` — batch danych (np. 64 próbek) |



> Czyli X to jest jeden tensor który zawiera dane dla 64 próbek i 8 cech (pojedyncza wartość w każdej tablicy tensora)

In [25]:
for batch in train_loader:
    X, y = batch
    print(len(batch))
    print(X)
    print('----')
    print(y)
    break  # dla pierwszego batcha

2
tensor([[ 4.9107e-01,  5.8485e-01,  8.7154e-02, -3.7258e-01, -2.8300e-01,
         -2.5591e-03, -7.8742e-01,  7.2858e-01],
        [ 1.3629e+00, -1.0838e+00,  9.6338e-01, -1.5745e-01, -5.0200e-01,
          7.7105e-03,  1.1415e+00, -1.2979e+00],
        [ 1.6386e-02,  5.0539e-01,  8.9874e-02, -1.9519e-01,  1.4970e-01,
          2.4957e-02, -6.8911e-01,  1.1578e+00],
        [-1.0536e+00,  1.1411e+00,  2.1742e-01, -3.3325e-02, -5.6911e-01,
         -1.9193e-02,  5.2818e-01, -9.9974e-02],
        [ 1.1592e-01,  1.8562e+00, -2.8303e-02,  2.0721e-01, -1.8233e-01,
          5.2943e-03, -6.9847e-01,  6.8366e-01],
        [-1.0680e+00, -1.2427e+00, -2.6610e-01, -8.7844e-02, -1.0389e+00,
         -7.5704e-02,  1.5395e+00, -1.5225e+00],
        [-7.8292e-01, -3.6864e-01,  8.6650e-02, -1.4969e-01, -6.1768e-01,
         -3.8662e-02,  1.8531e+00, -1.0833e+00],
        [ 1.0081e+00, -1.5605e+00,  2.8017e-02, -2.1821e-01, -4.3930e-01,
          5.1921e-03, -9.1384e-01,  8.8831e-01],
        [-5.16

Dla każdej epoki:
  🔁 Trenuj na każdym batchu:
     - Oblicz stratę
     - Wyzeruj gradienty
     - Propagacja wsteczna
     - Aktualizacja wag
  ✅ Oblicz średnią stratę treningową

  🔁 Walidacja na każdym batchu:
     - Predykcja
     - Strata walidacyjna
  ✅ Oblicz średnią stratę walidacyjną

  🧾 Zapisz train_loss i val_loss do history

  🖨️ Co 10 epok – wypisz wynik


<a id="model-def-cnn"></a>
# <a id='toc10_'></a>[<b><span style='color:darkorange'>Step 1.2 |</span><span style='color:#1E3A8A'> Model definition</span></b>](#toc0_)

To nasz **główny model regresyjny** – czyli sieć neuronowa do przewidywania cen mieszkań.  
Dziedziczy po `nn.Module`, czyli bazowej klasie PyTorch do budowy modeli.



#### ✅ Co robi `__init__()`?

-   Tworzy warstwy modelu
    
-   Tworzy i zapisuje funkcję straty (domyślnie `nn.MSELoss()`)



✅ `forward()` jest wywoływana **automatycznie**, kiedy wywołasz model jak funkcję: `output = model(input)`


In [26]:
class CaliforniaHousingMLP(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)


    def epoch_end(self, epoch, train_loss, val_loss, max_epochs):
        if (epoch + 1) % 10 == 0 or epoch == max_epochs - 1:
            print(
                f"Epoch [{epoch+1}/{max_epochs}], | " 
                f"train_loss: {train_loss:.4f}, | " 
                f"val_loss: {val_loss:.4f}"
            )


### ✅ 1. **Czym jest graf obliczeniowy i dlaczego `validation_step()` nie oblicza gradientów?**

W PyTorch:

-   **każda operacja matematyczna** na tensorkach (np. dodawanie, mnożenie) jest zapisywana jako **węzeł** w tzw. **grafie obliczeniowym**.
    
-   ten graf służy do obliczania gradientów (czyli tzw. **backpropagation**).
    
```python
x = torch.tensor(2.0, requires_grad=True)
y = x * 3 + 1
```

Tu PyTorch:

-   zapamiętuje, że `y` zależy od `x`
    
-   tworzy graf obliczeniowy:  
    `x → (x * 3) → (+1) → y`
    

Teraz:

`y.backward()  # oblicza ∂y/∂x`

I masz gradient! 🎯



### Rozwiązanie: `.detach()`

👉 **Odłącza tensor od grafu**:


`loss = self.loss_fn(pred, y) return loss.detach()  # ← nie jest już częścią grafu`

**lub**:

`with torch.no_grad():     ...`

Obie metody mówią:  
🧼 "Nie twórz grafu obliczeniowego — tylko przelicz wynik."



### ✅ 2. **Czemu używamy `torch.stack()` w `validation_epoch_end()`?**


`torch.stack()` Zamienia listę tensorów w jeden większy tensor.


`val_losses = [tensor(0.3), tensor(0.4), tensor(0.5)]  # to są oddzielne tensory — nie możesz obliczyć na nich .mean()`

A więc robimy:

`torch.stack(val_losses) # => tensor([0.3, 0.4, 0.5])`

Dzięki temu można:

`torch.stack(val_losses).mean()`

Czyli obliczyć **średnią stratę walidacyjną z epoki**.



### ✅ 3. **Dlaczego w `epoch_end()` mamy taki warunek:**

Wydrukuj tylko:

-   co **10 epok**
    
-   oraz **na sam koniec** treningu (ostatnia epoka)
    

Żeby nie spamować terminala:

-   Jeśli masz np. `100 epok`, nie chcesz widzieć 100 linijek
    
-   Wystarczy co 10 epok + na końcu



<a id="training-both-models"></a>
# <a id='toc10_'></a>[<b><span style='color:darkorange'>Step 1.3 |</span><span style='color:#1E3A8A'> Training model</span></b>](#toc0_)

## 🎓 **Co to jest epoka (epoch)?**

> ✅ **Epoka to jeden pełny „przegląd” całego zbioru treningowego przez model.**

Czyli: 👉 Model widzi **każdy przykład raz**  
👉 Liczy stratę  
👉 Uczy się i aktualizuje wagi


## 🧠 **Analogia: nauka z podręcznika**

Wyobraź sobie, że uczysz się z podręcznika z 1000 pytań.

-   ✅ **Epoka** = przeczytałeś i przećwiczyłeś **wszystkie 1000 pytań raz**
    
-   🔁 **Druga epoka** = znowu przeczytałeś te same pytania (bo chcesz się poprawić)
    
-   🔁 Trzecia epoka = kolejna powtórka itd.
    

Im więcej epok:

-   tym lepiej zapamiętujesz
    
-   ale... możesz też zacząć **uczyć się na pamięć (overfitting)** ❗



### `train_loop(...)`


Trenuje model przez **jedną epokę** na wszystkich batchach z `train_loader`.


1.  `model.train()` – mówi PyTorchowi: „jestem w trybie treningu” (np. dropout działa)
    
2.  Dla każdego `batch`:
    
    -   liczy stratę: `loss = model.training_step(batch)`
        -   pred = model(x)      <---> forward
        
        - loss = loss_fn(pred, y)  <---> oblicz stratę
        
    -   oblicza gradienty: `loss.backward()`
        
    -   aktualizuje wagi: `optimizer.step()`
        
    -   zeruje gradienty: `optimizer.zero_grad()` Czyści gradienty z poprzedniego kroku, zanim obliczysz nowe gradienty dla aktualnego batcha.
        
3.  Zwraca średnią stratę



In [ ]:
# przykład bez zero_grad
# iteracja 1
loss1.backward()  # ∂L1/∂W → zapisuje gradient
# iteracja 2
loss2.backward()  # ∂L2/∂W → DODAJE się do poprzedniego!


### `train_model_flexible(...)`


To **główna funkcja treningowa**.  
Steruje całą nauką modelu przez wiele epok, z opcjami jak:

-   early stopping
    
-   weight decay
    
-   scheduler
    

1.  Inicjalizuje optimizer (np. Adam)
    
2.  (Opcjonalnie) scheduler
    
3.  Dla każdej epoki:
    
    -   `train_loop(...)`
        
    -   `model.validation_step(...)`
        
    -   `model.validation_epoch_end(...)`
        
    -   logowanie `epoch_end(...)`
        
    -   (opcjonalnie) early stopping
        
    -   (opcjonalnie) scheduler.step()
        

📦 Na końcu zwraca `history` (dane do wykresów: train/val loss)



In [38]:
# Pętle treningowe
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    total_loss = []
    for batch in dataloader:        #  for batch, (X, y) in enumerate(dataloader):
        X, y = batch                #   X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()

        # Gradient diagnostyka (1. batch)
        if batch == 0:
            for name, param in model.named_parameters():
                if param.requires_grad and param.grad is not None:
                    print(f"[{name}] grad mean: {param.grad.mean():.6f}, std: {param.grad.std():.6f}")
                    break

        optimizer.step()
        total_loss.append(loss.item())
    return np.mean(total_loss)

def test_loop(dataloader, model, loss_fn):
    model.eval()
    total_loss = []
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            loss = loss_fn(pred, y)
            total_loss.append(loss.item())
    return np.mean(total_loss)

In [39]:
def train_model_flexible(model, name, train_loader, val_loader, loss_fn,
                         lr=1e-3, weight_decay=False, early_stopping=False,
                         scheduler=False, patience=3, max_epochs=50):

    # ➕ Regularyzacja
    wd = 1e-2 if weight_decay else 0.0
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

    # ➕ Scheduler
    sched = None
    if scheduler:
        sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

    history = {'train_loss': [], 'val_loss': []}
    best_loss = float('inf')
    stop_counter = 0

    print(f"\n🧠 Training {name}...\n" + "-" * 50)

    for epoch in range(max_epochs):
        train_loss = train_loop(train_loader, model, loss_fn, optimizer)
        val_loss = test_loop(val_loader, model, loss_fn)

        if sched:
            sched.step(val_loss)

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)

        model.epoch_end(epoch, train_loss, val_loss, max_epochs)

        if early_stopping:
            if val_loss < best_loss:
                best_loss = val_loss
                stop_counter = 0
            else:
                stop_counter += 1
                if stop_counter >= patience:
                    print("⏹️ Early stopping triggered.")
                    break

    return history


In [40]:
# Trening modelu
epochs = 100
learning_rate = 1e-3
loss_fn = nn.MSELoss()


model = CaliforniaHousingMLP(input_size=X.shape[1])
history = train_model_flexible(
    model,
    name="CaliforniaHousingMLP",
    train_loader=train_loader,
    val_loader=val_loader,
    loss_fn=loss_fn,
    lr=learning_rate,
    early_stopping=False,
    scheduler=False,
    max_epochs=epochs
)


🧠 Training CaliforniaHousingMLP...
--------------------------------------------------
Epoch [10/100], | train_loss: 0.3226, | val_loss: 0.3324
Epoch [20/100], | train_loss: 0.2908, | val_loss: 0.3012
Epoch [30/100], | train_loss: 0.2778, | val_loss: 0.2894
Epoch [40/100], | train_loss: 0.2713, | val_loss: 0.2926
Epoch [50/100], | train_loss: 0.2655, | val_loss: 0.2838
Epoch [60/100], | train_loss: 0.2615, | val_loss: 0.2906
Epoch [70/100], | train_loss: 0.2564, | val_loss: 0.2847
Epoch [80/100], | train_loss: 0.2531, | val_loss: 0.2806
Epoch [90/100], | train_loss: 0.2506, | val_loss: 0.2733
Epoch [100/100], | train_loss: 0.2467, | val_loss: 0.2743


<a id="plots-loss-accuracy"></a>
# <a id='toc10_'></a>[<b><span style='color:darkorange'>Step 1.4 |</span><span style='color:#1E3A8A'> Loss & Accuracy</span></b>](#toc0_)

Wykres strat treningowych i walidacyjnych (loss vs epoch) – w Plotly.



In [33]:
# Wykresy
def plot_training_curves_regression(history):
    epochs_range = list(range(1, len(history['train_loss']) + 1))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=epochs_range, y=history['train_loss'], mode='lines+markers', name='Train Loss'))
    fig.add_trace(go.Scatter(x=epochs_range, y=history['val_loss'], mode='lines+markers', name='Validation Loss'))
    fig.update_layout(
        title="📉 Loss during Training (Regression)",
        xaxis_title="Epoch",
        yaxis_title="MSE Loss",
        template="plotly_white"
    )
    fig.show()

In [41]:
# Wykres strat
plot_training_curves_regression(history)

# Wykres predykcji vs rzeczywistość
model.eval()
with torch.no_grad():
    predictions = model(X_val_tensor).squeeze().numpy()
    actuals = y_val_tensor.squeeze().numpy()

scatter_fig = go.Figure()
scatter_fig.add_trace(go.Scatter(
    x=actuals, y=predictions, mode='markers', name='Predictions'
))
scatter_fig.add_trace(go.Scatter(
    x=[actuals.min(), actuals.max()],
    y=[actuals.min(), actuals.max()],
    mode='lines', name='Ideal', line=dict(dash='dash', color='red')
))
scatter_fig.update_layout(
    title="🏡 Actual vs Predicted House Prices",
    xaxis_title="Actual Price",
    yaxis_title="Predicted Price",
    template="plotly_white"
)
scatter_fig.show()